# 自动微分

## 1.标量变量的反向传播（y=2x^2）

In [36]:
import torch
# requires_grad 属性为 True，意味着 PyTorch 会追踪与这个张量相关的操作，允许计算该张量的梯度。
x = torch.tensor([0,1,2,3], dtype=torch.float32, requires_grad = True)
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [37]:
# y=2xTx,且是个标量
y = 2*torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [38]:
# 调用反向传播函数自动计算y关于x每个分量的梯度
y.backward()
print(x.grad)

tensor([ 0.,  4.,  8., 12.])


In [39]:
# 在默认情况，Pytorch会累积梯度，需要清楚之前的值
x.grad.zero_()
x.grad

tensor([0., 0., 0., 0.])

In [40]:
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 2. 非标量变量的反向传播

当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。 对于高阶和高维的y和x，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括[深度学习中]）， 但当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。 这里(，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。)

In [44]:
x.grad.zero_()
# 将x对应位置的元素平方得到新的张量
y = x*x
# 方法一：求和操作（sum()） 将一个张量中的所有元素加和，得到一个标量，从而允许反向传播。
# 通过将输出张量求和，我们可以确保反向传播的结果仍然是有意义的（即每个元素都有对应的梯度）。
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [45]:
x.grad.zero_()
# 将x对应位置的元素平方得到新的张量
y = x*x
# 方法二：为非标量的张量显式指定梯度
# 为每个元素显式指定梯度（与 y 形状相同）
gradients = torch.ones_like(y)
# 反向传播
y.backward(gradient=gradients)
x.grad

tensor([0., 2., 4., 6.])

## 3. 分离计算

将某些计算移动到记录的计算图之外，用于将参数固定

In [46]:
x.grad.zero_()
y = x * x
u = y.detach() # 将y作为常数，而不是关于x的函数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

## 4. 控制流的梯度计算

In [47]:
def f(a):
    b = a * 2
    # b.norm() 计算张量 b 的范数（默认为 L2 范数，即平方和的平方根）。
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = b + 100
    return c

In [50]:
a = torch.randn(size=(), requires_grad = True)
d = f(a)
d.backward()

In [52]:
# 因为f在其输入a中是分段线性的，因此对于任意一个a，都存在某一个常量标量k，使得f(a)=ka，即f(a)对a求偏导为f(a)/a
a.grad == d/a

tensor(True)